In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.datasets import fashion_mnist # fashion mnist데이터 셋 불러오기
import tensorflow.keras.layers as Layer
import numpy as np
import pandas as pd
import tensorflow as tf
import keras, os

Early stopping을 활용한 신경망 모델구현해보기

- overfitting을 방지하기 위한 Early stopping을 활용하여 신경망 모델 구현

In [22]:
# 데이터 불러오기
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape, x_test.shape)

# 데이터 정규화
x_train = x_train / 255
x_test = x_test / 255

# 클래스 확인
np.unique(y_train)

(60000, 28, 28) (10000, 28, 28)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [23]:
# 기본 신경만 만들기

# 모델 구성 확인
model = Sequential([
  Flatten(input_shape = (28, 28)), # 28 x 28 이미지 데이터를 펴주기 위해 Flatten사용
  Dense(10, activation= 'softmax') # 클래스가 10개이므로 10개의 노드 생성
])

# 컴파일 하기
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [24]:
# 모델 학습

# 변수 설정 (soft coding 습관화)
batch_size = 30
epochs_max = 1

# 학습시킨 데이터를 저장시키기 위한 코드
checkpoint_filepath = 'FMbest.hdf5'

# overfitting을 방지하기 위해서 학습 중 early stop을 실시해 주기 위한 코드 저장
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # 모니터링 하는 방법
    min_delta = 0,  # 지정 숫자보다 작으면 조기종료 발생(min_delta < 0 은 값이 증가할때 조기종료)
    patience=10, # 조기종료가 발생 후에 몇 번 더 epoch를 진행 할지
    verbose=1 # 자세한 정보 제공 (1이면 제공)
    ) 

# Validation Set을 기준으로 가장 최적의 모델을 찾는 코드
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath, monitor = 'val_loss', verbose = 1, 
    save_best_only = True, save_weights_only = True, mode = 'auto', save_freq = 'epoch',
    option = None
)

# 모델 학습 코드 + early stop + Best model
model.fit(x_train, y_train, batch_size=batch_size, epochs= epochs_max,
          verbose = 1, validation_data = (x_test, y_test),
          callbacks = [early_stop, save_best])

2000/2000 [==============================] - 3s 2ms/step - loss: 0.6011 - accuracy: 0.7972 - val_loss: 0.5113 - val_accuracy: 0.8227

Epoch 00001: val_loss improved from inf to 0.51133, saving model to FMbest.hdf5


In [25]:
# 학습된 모델을 이용하여 테스트하는 코드

model.predict(x_test[0:1])
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.5113 - accuracy: 0.8227


In [26]:
# 체크포인트에 저장된 가중치들을 불러들이는 코드

model.load_weights(checkpoint_filepath)

In [27]:
# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드

model.predict(x_test[0:1])
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1)

313/313 [==============================] - 0s 1ms/step - loss: 0.5113 - accuracy: 0.8227


Weight Decay 가중치 감소시키기

- 학습과정에서 overfitting을 방지하기 위한 기술, Regularization의 한 종류인 Weight Decay를 활용한 신경망 구현하기
- L1norm 과 L2 norm활용
- costfunction에 람다를 더해줌으로서 비용함수가 더큰거처럼보여 가중치를 감소하게 만들기 위해 학습시키는 방법

In [28]:
# Weight Decay를 전체적으로 반영한 예시 코드
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras import regularizers

# 모델 구성을 확인
model = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(64,
        kernel_regularizer = regularizers.l2(0.01), # L2 norm regularization
        activity_regularizer = regularizers.l1(0.01)),
        Dense(10, activation='softmax') # L1 norm regularization
])

# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics =['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size= 30, epochs = 1, verbose = 1, 
          validation_data = (x_test, y_test)) 


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 4s 2ms/step - loss: 1.0218 - accuracy: 0.7980 - val_loss: 0.8155 - val_accuracy: 0.7988


Constraints 를 활용한 신경망 구현하기
- overfitting을 방지하기 위해 강제로 가중치의 상한선을 지정해 그 이상은 일정한 값으로 치환시킨다.

In [29]:
# 모델 구성하기
model = Sequential([
                    Flatten(input_shape = (28, 28)),
                    Dense(64, kernel_regularizer = regularizers.l2(0.01),
                          activity_regularizer=regularizers.l1(0.01),
                          kernel_constraint = MaxNorm(2.)), # add constraints
                    Dense(10, activation = 'softmax') 
])

# 업데이트 방식을 설정합니다.
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()

model.fit(x_train, y_train, 
          batch_size= 30, epochs = 1,
          verbose = 1, validation_data =(x_test, y_test))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 1.0296 - accuracy: 0.7947 - val_loss: 0.8089 - val_accuracy: 0.8098


Dropout 을 활용한 신경망 구현하기
- dropout을 사용하면 overfitting을 방지할 수 있다.
- 일부 노드를 학습할때에만 차단 해줌으로서 일부 노드가 없이 결과를 예측하도록 만들고, 학습을 진행하기 때문에 과적합을 어느정도 차단할 수 있다. 즉, 노드를 임시로 몇개를 차단하여 강하게 키우는 느낌, test할때는 다시 노드를 사용하며, 랜덤으로 노드를 차단한다.

In [30]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras import regularizers

# 모델 구성을 확인합니다.
model = Sequential([
  Flatten(input_shape =(28,28)),
  Dense(64,
        kernel_regularizer = regularizers.l2(0.01),
        activity_regularizer = regularizers.l1(0.01),
        kernel_constraint=MaxNorm(2.))    ,
  Dropout(0.5),
  Dense(10, activation = 'softmax')
])

# 업데이트 방식을 설정합니다.
model.compile(optimizer='adam'
             , loss='sparse_categorical_crossentropy'
             , metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size = 30, epochs =1, verbose = 1, 
          validation_data = (x_test, y_test))


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                50240     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
2000/2000 [==============================] - 5s 2ms/step - loss: 1.2124 - accuracy: 0.7677 - val_loss: 0.8928 - val_accuracy: 0.8067


In [31]:

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

kf = KFold(n_splits = 5)             
skf = StratifiedKFold(n_splits = 5, random_state = 100, shuffle = True) 

x_train.shape


(60000, 28, 28)

In [34]:
for train_index, val_index in kf.split(np.zeros(x_train.shape[0]), y_train):
  print(train_index)

[12000 12001 12002 ... 59997 59998 59999]
[    0     1     2 ... 59997 59998 59999]
[    0     1     2 ... 59997 59998 59999]
[    0     1     2 ... 59997 59998 59999]
[    0     1     2 ... 47997 47998 47999]
